Installing required resources
----------------

In [ ]:
# Using pip3 to install for Python 3.x; remove the quiet flag (-q) if you want to see the output
!pip3 -q install numpy cython scipy sqlalchemy pandas matplotlib pymatbridge --user
!pip3 -q install git+https://github.com/christian-oreilly/spinkyDemo.git --user --upgrade
!pip3 -q install git+https://bitbucket.org/christian_oreilly/spyndle.git --user --upgrade

# This will throw an error when re-executed a second time because the SPINKY folder will already exist.
!git clone https://github.com/TarekLaj/SPINKY.git

In [1]:
# Imports and constants
import numpy as np
from scipy.io import savemat 
import sys
import os
from glob import glob

from spyndle.io import EDFReader
#from spyndle.detector import DetectorEvaluator

dbPath     = '/home/oreilly/MASS/'  # Path where MASS is stored
spinkyPath = os.path.join(os.getcwd(), 'SPINKY')

Starting Matlab and adding necessary paths to Matlab environement variable
--------

In [3]:
from pymatbridge import Matlab
mlab = Matlab(maxtime=60) #Matlab(matlab='C:/Program Files/MATLAB/R2015a/bin/matlab.exe')
mlab.start()

matlabCode = \
"rootPath = '" + spinkyPath + "/'; \n" + """
addpath(rootPath) 
addpath([rootPath 'functions'])
addpath([rootPath 'functions/tqwt_matlab_toolbox/tqwt_matlab_toolbox'])
"""
result = mlab.run_code(matlabCode)

/home/oreilly/.local/lib/python3.4/site-packages/IPython/nbformat.py:13: ShimWarning: The `IPython.nbformat` package has been deprecated. You should import from nbformat instead.
  "You should import from nbformat instead.", ShimWarning)


Starting MATLAB on ZMQ socket ipc:///tmp/pymatbridge-e0b19916-9a50-4922-810b-b905374b0fa5
Send 'exit' command to kill the server
............MATLAB started and connected!


Example of threshold estimation (for K-complexes detection)
---------------

In [ ]:
from spinkyDemo import training_process, getTrainSignal, kp_thresholds_ranges

mode        = 'kcomplex' #'spindles'
strSubject  = "01"
scorer      = 'kComplexE1'
nbTrainPage = 30

reader = EDFReader(dbPath + '01-02-00' + strSubject + ' PSG.edf', 
                   annotationFileName=[dbPath + '01-02-00' + strSubject + ' Base.edf',
                                       dbPath + '01-02-00' + strSubject + ' KComplexesE1.edf'])

trainSig, nbEvents, fs, epoch_length = getTrainSignal(reader, 
                                                      nbTrainPage=nbTrainPage, 
                                                      eventName=scorer)

threshold = kp_thresholds_ranges(mlab, trainSig, epoch_length)[0]

savemat('temp_train.mat', {'trainSig':trainSig, 'fs':fs, 'epoch_length':epoch_length, 
                           'threshold':thresh, 'mode':mode, 'nbEvents':nbEvents})            

threshold = training_process(mlab, trainSig, fs, epoch_length, mode, threshold, nbEvents)

print(strSubject, scorer, threshold)            

Example of K-Complex detection
---------------

In [ ]:
from spinkyDemo import getTestSignal

reader = EDFReader(dbPath + '01-02-00' + strSubject + ' PSG.edf', 
                   annotationFileName=[dbPath + '01-02-00' + strSubject + ' Base.edf'])

testSig, fs, epoch_length = getTestSignal(reader)
    
subject = '01-02-00' + strSubject + "_" + scorer + "_" + str(threshold)

# Due to an apparent bug in python-matlab-bridge, the program hang on 
# call to mlab.set_variable with numpy array that are too large as in
# testSig. Thus, we need to get around this limitation by saving the arguments
# in a mat file and calling an wrapper matlab function which will read this mat
# file and call test_process
#nbr_sp, pos_sp = test_process(mlab, testSig, fs, epoch_length, subject, threshold, mode)

savemat('temp.mat', {'testSig':testSig, 'fs':fs, 'epoch_length':epoch_length, 
                     'subject':subject, 'threshold':threshold, 'mode':mode})

result = mlab.run_code("test_process_wrapper('temp.mat');")

print(result["content"]["stdout"])

Example of performance assessment
---------------

In [ ]:
def spindleInPage(spindle, page):
    return spindle.startTime >= page.startTime and spindle.startTime <= page.startTime + page.duration()

def makeIndexSignal(spindles, epoch_length, fs):    
    index = np.zeros((int(epoch_length*fs), 1))
    for s, e in spindles:
        index[int(np.round(s*fs)):int(np.round(e*fs))] = 1
    return index
    
    
def compute2classComp(indGold, indTest):    
    
    TP = np.logical_and(indGold, indTest)
    TN = np.logical_and(np.logical_not(indGold), np.logical_not(indTest)) 
    FP = np.logical_and(np.logical_not(indGold), indTest)
    FN = np.logical_and(indGold, np.logical_not(indTest))

    return TP, TN, FP, FN


def prinResults(threshold, percentile, subject, scorer, TP, TN, FP, FN):
    res = (subject, percentile, threshold, scorer,
            sensitivity(TP, TN, FP, FN), 
            specificity(TP, TN, FP, FN),
            PPV(TP, TN, FP, FN), 
            MCC(TP, TN, FP, FN),
            cohenk(TP, TN, FP, FN), 
            F1(TP, TN, FP, FN))
    print(("Subject:%s, percentile:%s, threshold:%s, scorer:%s, "\
           "sensitivity=%f, specificity=%f, PPV=%f,  MCC=%f, CohenK=%f, F1=%f" % res))     
    return res

    
channel      = 'EEG C3-CLE'
pathResFiles = 'C:/Users/oreichri/Dropbox/Publications (current)/PaperDetectorTarekKarim/CK/'
evaluator = DetectorEvaluator()

reader = EDFReader(dbPath + '01-02-00' + strSubject + ' PSG.edf', 
                   annotationFileName=[dbPath + '01-02-00' + strSubject + ' Base.edf',
                                       dbPath + '01-02-00' + strSubject + ' KComplexesE1.edf'])

pages  = [e for e in reader.events if e.name == 'Sleep stage 2']
fs = reader.getChannelFreq(channel)
testSig   = []
epoch_length = np.min([s.duration() for s in pages])
nbSamples = int(epoch_length*fs)

files = glob(pathResFiles + '*_01-02-00' + strSubject + "_" + scorer + "_*.txt")
if len(files):
    spindles = [e for e in reader.events if e.name == scorer]

for resFileName in files:
    detectedSpindles = readDetectorOutput(resFileName)

    tokens = resFileName.split("_")
    threshold = float(tokens[-1].split(".")[0])
    percentile = int(tokens[-2])
    subject = int(tokens[-4].split("-")[-1])
    TP = []
    TN = [] 
    FP = []
    FN = []
    for noPage in np.unique(detectedSpindles["page"]):
        p = pages[noPage-1]
        spindlePage = [(s.startTime - p.startTime, s.startTime + s.duration() - p.startTime) for s in spindles 
                                       if spindleInPage(s, p)]
        spindlePage = []
        for s in spindles:
            if spindleInPage(s, p):
                sig = reader.read([channel], s.startTime,  s.duration())[channel].signal
                pic = s.startTime - p.startTime + float(np.argmin(sig))/fs
                spindlePage.append((pic-0.1, pic+1.3))

        detectSpindlePage = [(l["time"]-0.1, l["time"]+1.3) for index, l 
                                 in detectedSpindles[detectedSpindles["page"] == noPage].iterrows()]

        indGold = makeIndexSignal(spindlePage, epoch_length, fs)
        indTest = makeIndexSignal(detectSpindlePage, epoch_length, fs)

        TPtmp, TNtmp, FPtmp, FNtmp = compute2classComp(indGold, indTest)
        TP.extend(TPtmp)
        TN.extend(TNtmp)
        FP.extend(FPtmp)
        FN.extend(FNtmp)

    prinResults(threshold, percentile, subject, scorer,
                np.sum(TP, dtype=np.int64), 
                np.sum(TN, dtype=np.int64), 
                np.sum(FP, dtype=np.int64), 
                np.sum(FN, dtype=np.int64))

Quitting Matlab
-----------------------

In [ ]:
mlab.quit()